In [ ]:
import pandas as pd
import numpy as np

try: # If you are running this notebook on your local machine
  # Put the path to your CSV file here
  file_name = "./Customerbehavior_Final.csv"
  df = pd.read_csv(file_name, index_col=0, low_memory=False)
except : # If you are running this notebook on Google Colab
  from google.colab import drive
  drive.mount('/content/drive')
  # Put the path to your CSV file here
  file_name = "./drive/MyDrive/Data-IA/Customerbehavior_Final.csv"
  df = pd.read_csv(file_name, index_col=0, low_memory=False)

df.head()

,StoreCode,BillCode,EnterTime,LeaveTime,BoxSize,MealType,OrderTime,MealSpending,BoxSpending,CustomerNumber,MaleNumber,FemaleNumber,Age,Membership
0,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Light Meal,2016-02-01 01:04:13,114,2857,6,3,3,30~39,0
1,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Project Package,2016-02-01 01:43:29,380,2857,6,3,3,30~39,0
2,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Beer,2016-02-01 02:17:11,280,2857,6,3,3,30~39,0
3,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Beverages (not including wine),2016-02-01 01:04:13,210,2857,6,3,3,30~39,0
4,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Fruit,2016-02-01 01:43:29,343,2857,6,3,3,30~39,0


In [ ]:
df.nunique()

StoreCode               8
BillCode           121101
EnterTime          225732
LeaveTime          209237
BoxSize                11
MealType               23
OrderTime         1725856
MealSpending         1128
BoxSpending          9584
CustomerNumber         71
MaleNumber             43
FemaleNumber           47
Age                     5
Membership              2
dtype: int64

## Apply complete store name

In [ ]:
def store_name(value):
    if value == 107: return 'SOGO Store'
    elif value == 108: return 'Tao Yuan'
    elif value == 112: return 'Jia yi'
    elif value == 113: return 'Kaohsiung'
    elif value == 114: return 'Zhong Hua Xin dian'
    else: return 'Lin Sheng'

df['StoreName'] = df['StoreCode'].apply(store_name)
df.head(5)

,StoreCode,BillCode,EnterTime,LeaveTime,BoxSize,MealType,OrderTime,MealSpending,BoxSpending,CustomerNumber,MaleNumber,FemaleNumber,Age,Membership,StoreName
0,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Light Meal,2016-02-01 01:04:13,114,2857,6,3,3,30~39,0,Lin Sheng
1,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Project Package,2016-02-01 01:43:29,380,2857,6,3,3,30~39,0,Lin Sheng
2,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Beer,2016-02-01 02:17:11,280,2857,6,3,3,30~39,0,Lin Sheng
3,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Beverages (not including wine),2016-02-01 01:04:13,210,2857,6,3,3,30~39,0,Lin Sheng
4,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Fruit,2016-02-01 01:43:29,343,2857,6,3,3,30~39,0,Lin Sheng


## Create UniqueID

In [ ]:
# Create "UniqueID" columns by concatenating StoreCode and BillCode
df["UniqueID"] = df['BillCode'].astype(str) + df['StoreCode'].astype(str)
df.head()

,StoreCode,BillCode,EnterTime,LeaveTime,BoxSize,MealType,OrderTime,MealSpending,BoxSpending,CustomerNumber,MaleNumber,FemaleNumber,Age,Membership,StoreName,UniqueID
0,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Light Meal,2016-02-01 01:04:13,114,2857,6,3,3,30~39,0,Lin Sheng,1602010011107
1,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Project Package,2016-02-01 01:43:29,380,2857,6,3,3,30~39,0,Lin Sheng,1602010011107
2,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Beer,2016-02-01 02:17:11,280,2857,6,3,3,30~39,0,Lin Sheng,1602010011107
3,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Beverages (not including wine),2016-02-01 01:04:13,210,2857,6,3,3,30~39,0,Lin Sheng,1602010011107
4,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Fruit,2016-02-01 01:43:29,343,2857,6,3,3,30~39,0,Lin Sheng,1602010011107


## Change date string to date pandas object

In [ ]:
# Create function to format the time stamp to date-time object
# def get_time(value):
#     temp = str.replace(value, "/", "-")
#     count1 = temp.count(":")
#     if count1 == 1: temp = temp + ":00"
#     return datetime.strptime(temp, "%Y-%m-%d %H:%M:%S")

# # Create new Time columns containing date-time object format
# df["TimeEnter"] = df["EnterTime"].apply(get_time)
# df["TimeLeave"] = df["LeaveTime"].apply(get_time)
# df["TimeOrder"] = df["OrderTime"].apply(get_time)

In [ ]:
# The methods below are more efficient than 'apply' as they utilize pandas vectorized operations
# for string manipulation and datetime conversion,
# which are optimized for performance on entire columns.
df['EnterTime'] = df['EnterTime'].str.replace("/", "-").str.pad(19, side='right', fillchar='0')
df['LeaveTime'] = df['LeaveTime'].str.replace("/", "-").str.pad(19, side='right', fillchar='0')
df['OrderTime'] = df['OrderTime'].str.replace("/", "-").str.pad(19, side='right', fillchar='0')

df["TimeEnter"] = pd.to_datetime(df["EnterTime"], format="%Y-%m-%d %H:%M:%S")
df["TimeLeave"] = pd.to_datetime(df["LeaveTime"], format="%Y-%m-%d %H:%M:%S")
df["TimeOrder"] = pd.to_datetime(df["OrderTime"], format="%Y-%m-%d %H:%M:%S")

In [ ]:
df["Duration(hours)"] = (df["TimeLeave"] - df["TimeEnter"]) / np.timedelta64(1, 's') / 3600
df.head()

,StoreCode,BillCode,EnterTime,LeaveTime,BoxSize,MealType,OrderTime,MealSpending,BoxSpending,CustomerNumber,MaleNumber,FemaleNumber,Age,Membership,StoreName,UniqueID,TimeEnter,TimeLeave,TimeOrder,Duration(hours)
0,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Light Meal,2016-02-01 01:04:13,114,2857,6,3,3,30~39,0,Lin Sheng,1602010011107,2016-02-01 00:49:00,2016-02-01 06:01:00,2016-02-01 01:04:13,5.2
1,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Project Package,2016-02-01 01:43:29,380,2857,6,3,3,30~39,0,Lin Sheng,1602010011107,2016-02-01 00:49:00,2016-02-01 06:01:00,2016-02-01 01:43:29,5.2
2,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Beer,2016-02-01 02:17:11,280,2857,6,3,3,30~39,0,Lin Sheng,1602010011107,2016-02-01 00:49:00,2016-02-01 06:01:00,2016-02-01 02:17:11,5.2
3,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Beverages (not including wine),2016-02-01 01:04:13,210,2857,6,3,3,30~39,0,Lin Sheng,1602010011107,2016-02-01 00:49:00,2016-02-01 06:01:00,2016-02-01 01:04:13,5.2
4,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Fruit,2016-02-01 01:43:29,343,2857,6,3,3,30~39,0,Lin Sheng,1602010011107,2016-02-01 00:49:00,2016-02-01 06:01:00,2016-02-01 01:43:29,5.2
